In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

### Plan

* start from all shapes for a single operator
* for each shape, assess for each hour of day x weekday/sat/sun (with day of week/time of day cols):
    * (can loop over weekday/sat/sun here at trip+stop_times join)
    * existing frequency at midpoint stop
    * current total runtime
* preserve route_id in main table
* (seperately) calculate operator/routes/shapes in each Census tract
* can then join tracts to service info; characterize route service target geographically
    * generate additional frequencies/service hours/service miles for service target
* (optional) generate hypothetical trips table
* (optional) assign service hours/miles to tracts (not sure why we'd need this yet)

In [51]:
def find_midpoint_stop(shape, trips, stop_times):
    return

In [45]:
weekday = (tbl.views.gtfs_schedule_fact_daily_trips()
     >> select(_.calitp_itp_id, _.service_date, _.trip_key, _.trip_id, _.is_in_service)
     >> filter(_.calitp_itp_id == 300)
     >> filter(_.service_date == '2021-10-07')
     >> filter(_.is_in_service == True)
     # >> show_query()
     >> collect())

In [46]:
saturday = (tbl.views.gtfs_schedule_fact_daily_trips()
     >> select(_.calitp_itp_id, _.service_date, _.trip_key, _.trip_id, _.is_in_service)
     >> filter(_.calitp_itp_id == 300)
     >> filter(_.service_date == '2021-10-09')
     >> filter(_.is_in_service == True)
     # >> show_query()
     >> collect())

In [47]:
sunday = (tbl.views.gtfs_schedule_fact_daily_trips()
     >> select(_.calitp_itp_id, _.service_date, _.trip_key, _.trip_id, _.is_in_service)
     >> filter(_.calitp_itp_id == 300)
     >> filter(_.service_date == '2021-10-10')
     >> filter(_.is_in_service == True)
     # >> show_query()
     >> collect())

In [48]:
weekday

,calitp_itp_id,service_date,trip_key,trip_id,is_in_service
0,300,2021-10-07,5972313532268407090,860329,True
1,300,2021-10-07,-4307186013627831998,860390,True
2,300,2021-10-07,2305409055359356341,860348,True
3,300,2021-10-07,-7850518669924244751,860354,True
4,300,2021-10-07,-1686891701138252615,859843,True
...,...,...,...,...,...
1479,300,2021-10-07,8069493734974821516,858450,True
1480,300,2021-10-07,-7643906475521810310,857186,True
1481,300,2021-10-07,-4214689238777897281,857161,True
1482,300,2021-10-07,8408740946127252926,857152,True


In [49]:
saturday

,calitp_itp_id,service_date,trip_key,trip_id,is_in_service
0,300,2021-10-09,5496780292698240775,857621,True
1,300,2021-10-09,-5873171032815797627,857599,True
2,300,2021-10-09,8391051661380735181,857617,True
3,300,2021-10-09,-9188172815247136275,857631,True
4,300,2021-10-09,6994791157854884369,857639,True
...,...,...,...,...,...
865,300,2021-10-09,-3889599468540500901,858364,True
866,300,2021-10-09,-1538584831610800853,856885,True
867,300,2021-10-09,-7063592539020574249,856937,True
868,300,2021-10-09,-4804762398391339110,856884,True


In [50]:
sunday

,calitp_itp_id,service_date,trip_key,trip_id,is_in_service
0,300,2021-10-10,-1566067371968892885,857665,True
1,300,2021-10-10,-2101857464276218548,857728,True
2,300,2021-10-10,-418902018110506047,857737,True
3,300,2021-10-10,-1625451368464181312,856247,True
4,300,2021-10-10,-9084693455007451867,856268,True
...,...,...,...,...,...
765,300,2021-10-10,-8389758290909726702,856982,True
766,300,2021-10-10,-5669199107012448594,856955,True
767,300,2021-10-10,-4817212099725408206,857050,True
768,300,2021-10-10,-8679058570687179377,857063,True


In [38]:
sunday.trip_id.unique()

array(['860329', '860390', '860348', ..., '857161', '857152', '857168'],
      dtype=object)

In [44]:
weekday['trip_id'].isin(sunday.trip_id).value_counts()

True    1484
Name: trip_id, dtype: int64

In [42]:
weekday.trip_id.is_unique

True

In [36]:
tbl.views.gtfs_schedule_dim_trips() >> filter(_.trip_key.isin(df.trip_key))

,calitp_itp_id,calitp_url_number,route_id,service_id,trip_id,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at,calitp_hash,trip_key,calitp_deleted_at
0,300,0,3329,10,856323,25320,UCLA,None,0,None,0,0,2021-09-01,SUNt6O6sEOlmqF7TDcy9aQ==,-6931215805910091774,2021-10-11
1,300,0,3338,10,859874,25352,Palms Rapid,None,1,None,0,0,2021-09-01,DfXpl4agd/nBkq4ssu3jvA==,5548948652349145677,2021-10-11
2,300,0,3328,10,856037,25314,UCLA,None,0,None,0,0,2021-09-01,QLrFr1A0KanMD9CfSOEgzg==,3991304789260610440,2021-10-11
3,300,0,3329,10,856342,25321,UCLA,None,0,None,0,0,2021-09-01,ugcELfTTZBLD6GZw8HX2Pg==,4194119154593412596,2021-10-11
4,300,0,3339,1,857922,25354,Brentwood,None,0,None,0,0,2021-09-01,vrGGszkkXyrjYHUy6kXHoA==,3893897006637506314,2021-10-11
